In [1]:
import torch
from torch.utils.data.dataset import Dataset
import torch.nn.functional as F
import torch.nn as nn
import torch.utils
import torch.distributions

from torchvision import transforms, utils
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
import pandas as pd
from dataloader import *
from model import *

In [9]:
#only on validation set
csv_path = 'progress.csv'
img_path = '/home/vip/sayan-mandal/datasets/obj_criteria/good_reduced/'

CDL = CLUnivDataLoader(csv_path, img_path, transform = transforms.Compose([transforms.Resize(64),transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),timestep=5,n=5)

#random sampler
batch_size = 64
validation_split = .2
test_split = .2
train_split = 1-validation_split-test_split
shuffle_dataset = True

# Creating data indices for training and validation splits:
dataset_size = len(CDL)
indices = list(range(dataset_size))
split_train = int(np.floor(train_split * dataset_size))
split_valid = int(np.floor((train_split+validation_split) * dataset_size))
if shuffle_dataset :
    np.random.shuffle(indices)
train_indices, val_indices, test_indices =  indices[:split_train],indices[split_train:split_valid],indices[split_valid:]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = torch.utils.data.DataLoader(CDL, batch_size=batch_size, sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(CDL, batch_size=batch_size, sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(CDL, batch_size=batch_size, sampler=test_sampler)

In [10]:
for i in range(10):
    print(CDL[i][0])

['AL0320_2005_02_16_OS.JPG', 'AL0320_2006_01_13_OS.JPG', 'AL0320_2007_03_26_OS.JPG', 'AL0320_2011_04_26_OS_02.JPG', 'AL0320_2012_07_25_OS_02.JPG']
['AL0320_2006_01_13_OS.JPG', 'AL0320_2007_03_26_OS.JPG', 'AL0320_2011_04_26_OS_02.JPG', 'AL0320_2012_07_25_OS_02.JPG', 'AL0320_2014_02_18_OS_02.JPG']
['AL0320_2007_03_26_OS.JPG', 'AL0320_2011_04_26_OS_02.JPG', 'AL0320_2012_07_25_OS_02.JPG', 'AL0320_2014_02_18_OS_02.JPG', 'AL0320_2015_03_19_OS_02.JPG']
['AL0320_2005_02_16_OD.JPG', 'AL0320_2006_01_13_OD.JPG', 'AL0320_2007_03_26_OD.JPG', 'AL0320_2011_04_26_OD_01.JPG', 'AL0320_2012_07_25_OD_02.JPG']
['AL0320_2006_01_13_OD.JPG', 'AL0320_2007_03_26_OD.JPG', 'AL0320_2011_04_26_OD_01.JPG', 'AL0320_2012_07_25_OD_02.JPG', 'AL0320_2014_02_18_OD_02.JPG']
['AL0320_2007_03_26_OD.JPG', 'AL0320_2011_04_26_OD_01.JPG', 'AL0320_2012_07_25_OD_02.JPG', 'AL0320_2014_02_18_OD_02.JPG', 'AL0320_2015_03_19_OD_01.JPG']
['AL0348_2004_05_05_OS.JPG', 'AL0348_2005_04_26_OS.JPG', 'AL0348_2007_04_30_OS.JPG', 'AL0348_2011_05

In [8]:
im=torch.zeros(3,128,128)

In [2]:
s = 64
c = 3
im1 = torch.rand(c,s,s)
im2 = torch.rand(c,s,s)
im3 = torch.rand(c,s,s)
im4 = torch.rand(c,s,s)
im5 = torch.rand(c,s,s)

In [5]:
im_w = torch.stack([im1,im2,im3,im4,im5],0)
im_w = torch.unsqueeze(im_w,0)
im_w.shape

torch.Size([1, 5, 3, 64, 64])

In [6]:
model = CNNLSTMNet(ts=5)
print(model)
o = model(im_w)

CNNLSTMNet(
  (ts_cnn): TimeDistributed(
    (layers): ModuleList(
      (0): CNNEncoder(
        (cnnmodule): Sequential(
          (0): ConvBlock(
            (seq): Sequential(
              (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
              (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU()
            )
          )
          (1): ConvBlock(
            (seq): Sequential(
              (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU()
            )
          )
          (2): ConvBlock(
            (seq): Sequential(
              (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
              (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU()
            )
          )

In [7]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [6]:
get_n_params(model)

11084801

In [8]:
get_n_params(model)

6809089

In [8]:
ts = 5
model = TimeDistributed(CNNEncoder, ts, 3)
model1 = LSTMModule(1024, 128, 2, ts, 2)

In [9]:
o = model(im_w)

xshape torch.Size([1, 64, 8, 8])
xshape torch.Size([1, 4096])
xshape torch.Size([1, 64, 8, 8])
xshape torch.Size([1, 4096])
xshape torch.Size([1, 64, 8, 8])
xshape torch.Size([1, 4096])
xshape torch.Size([1, 64, 8, 8])
xshape torch.Size([1, 4096])
xshape torch.Size([1, 64, 8, 8])
xshape torch.Size([1, 4096])
out torch.Size([1, 5, 512])


In [147]:
o.shape

torch.Size([1, 5, 1024])

In [148]:
o = model1(o)

In [149]:
o

tensor([[-0.0596, -0.0491]], grad_fn=<AddmmBackward>)

In [134]:
1024/4

256.0